# Importing Libraries

In [18]:
from matplotlib.ticker import FuncFormatter
from pandas.io.json import json_normalize
from urllib.request import urlopen,Request
import json
import locale
import matplotlib
import matplotlib.pyplot as plt
import os
import pandas as pd
import requests
import sqlalchemy
import sqlite3
import csv
%matplotlib inline
from IPython.display import Image, HTML
import datetime
import ast
import seaborn as sns
import numpy as np
from scipy import stats
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.model_selection import train_test_split
#from xgboost import XGBClassifier, XGBRegressor
#from wordcloud import WordCloud, STOPWORDS
#import plotly
#import plotly.plotly as py
#plotly.tools.set_credentials_file(username='rounakbanik', api_key='xTLaHBy9MVv5szF4Pwan')

sns.set_style('whitegrid')
sns.set(font_scale=1.25)
pd.set_option('display.max_colwidth', 50)


# Collecting and cleaning data sets

# Collecting

## Movies

In [21]:
def movies(directory, headers, api_key, stop, start, writing_mode):
    
    if not os.path.exists(directory):
        os.makedirs(directory)
        os.chdir(directory)
    else:
        os.chdir(directory)
    print(os.getcwd())
        
    while start < stop:
        response = requests.get('https://api.themoviedb.org/3/movie/' + str(start) + '?api_key=' + api_key)
        data = response.json()
        filtered_data = {k : v for k, v in filter(lambda t: t[0] in headers, data.items())}
        df = pd.DataFrame(filtered_data, index = [0])
                 
        with open('movies.csv', str(writing_mode)) as f:
            df.to_csv(f, header = False, sep = ',', encoding = 'utf-8')
        print(start)
        start += 1
        
    header_to_pass = ["idx"] + headers
    
    df = pd.read_csv(directory + 'movies.csv', header = None, names = header_to_pass, error_bad_lines = False)
    df.rename(columns={0: header_to_pass[0], 1: header_to_pass[1], 2: header_to_pass[2], 3: header_to_pass[3], 4: header_to_pass[4], 5: header_to_pass[5], 6: header_to_pass[6], 7: header_to_pass[7], 8: header_to_pass[8], 9: header_to_pass[9], 10: header_to_pass[10], 11: header_to_pass[11], 12: header_to_pass[12]}, inplace=True)
    df.to_csv(directory + '/backup/movies.csv', index = False)

    with open(directory + '/backup/movies.csv', 'r') as table:
        dr = csv.DictReader(table, delimiter = ',')
        to_db = [(i[header_to_pass[0]], i[header_to_pass[1]], i[header_to_pass[2]], i[header_to_pass[3]], i[header_to_pass[4]] , i[header_to_pass[5]], i[header_to_pass[6]], i[header_to_pass[7]], i[header_to_pass[8]], i[header_to_pass[9]], i[header_to_pass[10]], i[header_to_pass[11]], i[header_to_pass[12]] ) for i in dr]

        
    con = sqlite3.connect(directory + 'movies.sqlite')
    cursor = con.cursor()
    
    cursor.execute('''
    DROP TABLE IF EXISTS movies
    ''')
    con.commit()
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS movies(idx STR, id STR, imdb_id STR, original_language STR, original_title STR, 
                  popularity FLOAT, release_date DATE, revenue FLOAT, runtime INT, status STR, title STR, vote_average FLOAT, vote_count INT)
    ''')
    con.commit()
    
    cursor.executemany("INSERT INTO movies VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?);", to_db)
    con.commit()
    con.close()


directory = '/home/konrad/movies/'
headers = ["id","imdb_id", "original_language", "original_title", "popularity","release_date","revenue","runtime","status","title","vote_average","vote_count"]
api_key = 'e4f5b3b79e2ab374455c1c1e1d0ca014'
stop = 198235
start = 198229
writing_mode = 'a'

movies(directory, headers, api_key, stop, start, writing_mode)

/home/konrad/movies
198229
198230
198231
198232
198233
198234


## Genres

In [22]:
def genres(directory, headers, api_key, stop, start, writing_mode):
    
    if not os.path.exists(directory):
        os.makedirs(directory)
        os.chdir(directory)
    else:
        os.chdir(directory)
    print(os.getcwd())
        
    while start < stop:
        response = requests.get('https://api.themoviedb.org/3/movie/' + str(start) +'?api_key=' +  api_key)
        data = response.json()
        
        if 'The resource you requested could not be found.' in data.values():
            start += 1
            
        else:            
            data_normalized = json_normalize(data, "genres", ["imdb_id"])
            df = pd.DataFrame(data_normalized, index = [0])
            
            with open('genres.csv', str(writing_mode)) as f:
                df.to_csv(f, header = False, sep = ',', encoding = 'utf-8')
            print(start)
            
        start += 1

    header_to_pass = ["idx"] + headers

    df = pd.read_csv(directory + 'genres.csv', header=None, names=header_to_pass, error_bad_lines=False)
    df.rename(columns={0: header_to_pass[0], 1: header_to_pass[1], 2: header_to_pass[2], 3: header_to_pass[3]}, inplace=True)
    df.to_csv(directory + '/backup/genres.csv', index=False)

    with open(directory + '/backup/genres.csv', 'r') as table:
        dr = csv.DictReader(table, delimiter=',')
        to_db = [(i[header_to_pass[0]], i[header_to_pass[1]], i[header_to_pass[2]], i[header_to_pass[3]]) for i in dr]

    
    con = sqlite3.connect(directory + 'movies.sqlite')
    cursor = con.cursor()
    
    cursor.execute('''
    DROP TABLE IF EXISTS genres
    ''')
    con.commit()

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS genres(idx STR, id STR, name STR, imdb_id STR)
    ''')
    con.commit()

    cursor.executemany("INSERT INTO genres VALUES (?,?,?,?);", to_db)
    con.commit()
    con.close()
    

    
directory = '/home/konrad/movies/'
headers = ["id", "name", "imdb_id"]
api_key = '69d2d11ea34cd0ff95c454c73786a6a7'
stop = 71935
start = 71930
writing_mode = 'a'

genres(directory, headers, api_key, stop, start, writing_mode)

/home/konrad/movies
71930
71931
71932
71933
71934


## Overview

In [23]:
def overview(directory, headers, api_key, stop, start, writing_mode):

    if not os.path.exists(directory):
        os.makedirs(directory)
        os.chdir(directory)
    else:
        os.chdir(directory)
    print(os.getcwd())
        
    while start < stop:
        response = requests.get('https://api.themoviedb.org/3/movie/'+ str(start) + '?api_key='+  api_key)
        data = response.json()
        filtered_data = {k : v for k, v in filter(lambda t: t[0] in headers, data.items())}
        df = pd.DataFrame(filtered_data, index=[0])
                 
        with open('overview.csv', str(writing_mode)) as f:
            df.to_csv(f, header=False, sep=',', encoding='utf-8')
        print(start)    
        start += 1
      
    header_to_pass = ["idx"] + headers

    df = pd.read_csv(directory + 'overview.csv', header=None, names=header_to_pass, error_bad_lines=False)
    df.rename(columns={0: header_to_pass[0], 1: header_to_pass[1], 2: header_to_pass[2], 3: header_to_pass[3]}, inplace=True)
    df.to_csv(directory + 'backup/overview.csv', index=False)

    with open(directory + 'backup/overview.csv', 'r') as table:
        dr = csv.DictReader(table, delimiter=',')
        to_db = [(i[header_to_pass[0]], i[header_to_pass[1]], i[header_to_pass[2]], i[header_to_pass[3]]) for i in dr]
    
    
    con = sqlite3.connect(directory + 'movies.sqlite')
    cursor = con.cursor()
    
    cursor.execute('''
    DROP TABLE IF EXISTS overview
    ''')
    con.commit()
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS overview(idx STR, id STR, imdb_id STR, overview STR)
    ''')
    con.commit()
    
    cursor.executemany("INSERT INTO overview VALUES (?,?,?,?);", to_db)
    con.commit()
    con.close()


directory = '/home/konrad/movies/'
headers = ["id","imdb_id", "overview"]
api_key = 'e4f5b3b79e2ab374455c1c1e1d0ca014'
stop = 25130
start = 25125
writing_mode = 'a'

overview(directory, headers, api_key, stop, start, writing_mode)

/home/konrad/movies
25125
25126
25127
25128
25129


## Production Countries

In [24]:
def productionCountries(directory, headers, api_key, stop, start, writing_mode):

    if not os.path.exists(directory):
        os.makedirs(directory)
        os.chdir(directory)
    else:
        os.chdir(directory)
    print(os.getcwd())
        
    while start < stop:
        response = requests.get('https://api.themoviedb.org/3/movie/'+ str(start) +'?api_key='+  api_key)
        data = response.json()
        
        if 'The resource you requested could not be found.' in data.values():
            start += 1
            
        else:
            data_normalized = json_normalize(data,"production_countries",["imdb_id"])
            df = pd.DataFrame(data_normalized, index = [0])

            with open('productioncountries.csv', str(writing_mode)) as f:
                df.to_csv(f, header = False, sep = ',', encoding = 'utf-8')
            print(start)
        
        start += 1

    df = pd.read_csv(directory + '/productioncountries.csv', header = None, names = headers, error_bad_lines = False)
    df.rename(columns={0: headers[0], 1: headers[1], 2: headers[2]}, inplace = True)
    df.to_csv(directory + 'backup/productioncountries.csv', index = False)

    with open(directory + '/backup/productioncountries.csv', 'r') as table:
        dr = csv.DictReader(table, delimiter = ',')
        to_db = [(i[headers[0]], i[headers[1]], i[headers[2]]) for i in dr]

    
    con = sqlite3.connect(directory + 'movies.sqlite')
    cursor = con.cursor()
    
    cursor.execute('''
    DROP TABLE IF EXISTS productioncountries
    ''')
    con.commit()
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS productioncountries(iso_3166_1 STR, name STR, imdb_id STR)
    ''')
    con.commit()
    
    cursor.executemany("INSERT INTO productioncountries VALUES (?,?,?);", to_db)
    con.commit()
    con.close()

directory = '/home/konrad/movies/'
headers = ["iso_3166_1", "name", "imdb_id"]
api_key = 'ab9bd0e209dd5c8bf42b028a63bc4e29'
stop = 199710
start = 199704
writing_mode = 'a'

productionCountries(directory, headers, api_key, stop, start, writing_mode)

/home/konrad/movies
199704
199705
199708
199709


##  Keywords

In [28]:
def keywords(directory, headers, api_key, stop, start, writing_mode):

    if not os.path.exists(directory):
        os.makedirs(directory)
        os.chdir(directory)
    else:
        os.chdir(directory)
    print(os.getcwd())
    
    con = sqlite3.connect(directory + 'movies.sqlite')
    cursor = con.cursor()
          
    df = pd.read_sql_query("SELECT imdb_id FROM (SELECT DISTINCT imdb_id FROM movies) WHERE imdb_id <> ''", con)
    l = df['imdb_id'].tolist()
    editedList = l[start:stop]

    for i in editedList:
        response = requests.get("https://api.themoviedb.org/3/movie/" + i + "/keywords?api_key=" + api_key)
        data = response.json()
        data_normalized = json_normalize(data, "keywords", ["id"], meta_prefix = "Movie.")
        df = pd.DataFrame(data_normalized)

        with open(directory + 'keywords.csv', str(writing_mode)) as f:
            df.to_csv(f, header = False, sep = ',', encoding = 'utf-8')
        print(start) 
        start += 1
        
    header_to_pass = ["idx"] + headers
    
    df = pd.read_csv(directory + 'keywords.csv', header = None, names = header_to_pass, error_bad_lines = False)
    df.rename(columns={0: header_to_pass[0], 1: header_to_pass[1], 2: header_to_pass[2]}, inplace = True)
    df.to_csv(directory + '/backup/keywords.csv', index = False)

    with open(directory + '/backup/keywords.csv','r') as table:
        dr = csv.DictReader(table, delimiter = ',')
        to_db = [(i[header_to_pass[0]], i[header_to_pass[1]], i[header_to_pass[2]]) for i in dr]
        
    
    cursor.execute('''
        DROP TABLE IF EXISTS keywords
    ''')
    con.commit()
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS keywords(idx STR, keywords STR, id STR)
    ''')
    con.commit()
    
    cursor.executemany("INSERT INTO keywords VALUES (?,?,?);", to_db)
    con.commit()
    con.close()
    

directory = '/home/konrad/movies/'
headers = ["keywords", "id"]
api_key = 'e4f5b3b79e2ab374455c1c1e1d0ca014'
stop = 7160
start = 7155
writing_mode = 'a'

keywords(directory, headers, api_key, stop, start, writing_mode)

/home/konrad/movies
7155
7156
7157
7158
7159


## Cast

In [33]:
def cast(directory, headers, api_key, stop, start, writing_mode):

    if not os.path.exists(directory):
        os.makedirs(directory)
        os.chdir(directory)
    else:
        os.chdir(directory)
    print(os.getcwd())
    
    con = sqlite3.connect(directory + 'movies.sqlite')
    cursor = con.cursor()
    
    df = pd.read_sql_query("SELECT imdb_id FROM (SELECT DISTINCT imdb_id FROM movies) WHERE imdb_id <> ''", con)
    l = df['imdb_id'].tolist()
    editedList = l[start:stop]

    for i in editedList:
        response = requests.get("https://api.themoviedb.org/3/movie/" + i + "/casts?api_key=" + api_key)
        data = response.json()
        data_normalized = json_normalize(data, "cast", ["id"], meta_prefix = "Movie.")
        df = pd.DataFrame(data_normalized)

        with open(directory + 'cast.csv', str(writing_mode)) as f:
            df.to_csv(f, header = False, sep = ',', encoding = 'utf-8')
        
        print(start)    
        start += 1
        
    header_to_pass = ["idx"] + headers

    df = pd.read_csv(directory + 'cast.csv', header = None, names = header_to_pass, error_bad_lines = False)
    df.rename(columns = {0: header_to_pass[0], 1:  header_to_pass[1], 2:  header_to_pass[2], 3:  header_to_pass[3], 4:  header_to_pass[4], 5:  header_to_pass[5], 6:  header_to_pass[6], 7:  header_to_pass[7], 8:  header_to_pass[8], 9:  header_to_pass[9]}, inplace=True)
    df.to_csv(directory + '/backup/cast.csv', index = False)

    with open(directory + '/backup/cast.csv','r') as table:
        dr = csv.DictReader(table, delimiter = ',')
        to_db = [(i[header_to_pass[0]], i[header_to_pass[1]], i[header_to_pass[2]], i[header_to_pass[3]], i[header_to_pass[4]], i[header_to_pass[5]], i[header_to_pass[6]], i[header_to_pass[7]], i[header_to_pass[8]], i[header_to_pass[9]]) for i in dr]


    cursor.execute('''
        DROP TABLE IF EXISTS cast
    ''')
    con.commit()
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS cast(idx STR, cast_id STR, character STR, credit_id STR, gender STR, id STR, name STR, "Movie.order" STR, profile_path STR, "Movie.id" STR)
    ''')
    con.commit()
    
    cursor.executemany("INSERT INTO cast VALUES (?,?,?,?,?,?,?,?,?,?);", to_db)
    con.commit()
    con.close()
    
        

directory = '/home/konrad/movies/'
headers = ["cast_id", "character", "credit_id", "imgenderdb_id", "id", "name", "Movie.order", "profile_path", "Movie.id"]
api_key = 'ab9bd0e209dd5c8bf42b028a63bc4e29'
stop = 20955
start = 20953
writing_mode = 'a'

cast(directory, headers, api_key, stop, start, writing_mode)

/home/konrad/movies
20953
20954


## Crew

In [32]:
def crew(directory, headers, api_key, stop, start, writing_mode):

    if not os.path.exists(directory):
        os.makedirs(directory)
        os.chdir(directory)
    else:
        os.chdir(directory)
    print(os.getcwd())
    
    con = sqlite3.connect(directory + 'movies.sqlite')
    cursor = con.cursor()
    
    df = pd.read_sql_query("SELECT imdb_id FROM (SELECT DISTINCT imdb_id FROM movies) WHERE imdb_id <> ''", con)
    l = df['imdb_id'].tolist()
    editedList = l[start:stop]

    for i in editedList:
        response = requests.get("https://api.themoviedb.org/3/movie/" + i + "/casts?api_key=" + api_key)
        data = response.json()
        data_normalized = json_normalize(data,"crew", ["id"], meta_prefix = "Movie.")
        df = pd.DataFrame(data_normalized)
                 
        with open('crew.csv', str(writing_mode)) as f:
            df.to_csv(f, header=False, sep=',', encoding='utf-8')
        
        print(start)    
        start += 1
        
    header_to_pass = ["idx"] + headers
    
    df = pd.read_csv(directory + 'crew.csv', header = None, names = header_to_pass, error_bad_lines = False)
    df.rename(columns = {0: header_to_pass[0], 1: header_to_pass[1] ,2: header_to_pass[2], 3: header_to_pass[3], 4: header_to_pass[4], 5: header_to_pass[5], 6: header_to_pass[6], 7: header_to_pass[7], 8: header_to_pass[8]}, inplace = True)
    df.to_csv(directory + '/backup/crew.csv', index = False)

    with open(directory + '/backup/crew.csv','r') as table:
        dr = csv.DictReader(table, delimiter=',')
        to_db = [(i[header_to_pass[0]], i[header_to_pass[1]], i[header_to_pass[2]], i[header_to_pass[3]], i[header_to_pass[4]], i[header_to_pass[5]], i[header_to_pass[6]], i[header_to_pass[7]], i[header_to_pass[8]]) for i in dr]

    
    cursor.execute('''
    DROP TABLE IF EXISTS crew
    ''')
    con.commit()
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS crew(idx STR, credit_id STR, department STR, gender STR, id STR, job STR, name STR, profile_path STR, "Movie.id" STR)
    ''')
    con.commit()
    
    cursor.executemany("INSERT INTO crew VALUES (?,?,?,?,?,?,?,?,?);", to_db)
    con.commit()
    
    

directory = '/home/konrad/movies/'
headers = ["credit_id", "department", "gender", "id", "job", "name", "profile_path", "Movie.id"]
api_key = 'ab9bd0e209dd5c8bf42b028a63bc4e29'
stop = 21000
start = 20998
writing_mode = 'a'
        
crew(directory, headers, api_key, stop, start, writing_mode)

/home/konrad/movies
20998
20999


/home/konrad/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3191: DtypeWarning: Columns (3,4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


# Cleaning

Quering tables from movies.sqlite data base gives an information about empty rows and duplicate entries which have been pulled out during desining and testing of ETL process.

# Movies Dataset

In [36]:
con = sqlite3.connect('/home/konrad/movies/movies.sqlite')
c = con.cursor()

## Basic characteristics of Movie table before cleaning

Tha shape of Movies table:

In [37]:
pd.read_sql_query("SELECT * FROM movies;", con).shape

(198520, 13)

Attributes of fields:

In [38]:
pd.read_sql_query("SELECT * FROM movies;", con).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198520 entries, 0 to 198519
Data columns (total 13 columns):
idx                  198520 non-null int64
id                   198520 non-null object
imdb_id              198520 non-null object
original_language    198520 non-null object
original_title       198520 non-null object
popularity           198520 non-null object
release_date         198520 non-null object
revenue              198520 non-null object
runtime              198520 non-null object
status               198520 non-null object
title                198520 non-null object
vote_average         198520 non-null object
vote_count           198520 non-null object
dtypes: int64(1), object(12)
memory usage: 19.7+ MB


First five rows of Movie table:

In [39]:
pd.read_sql_query("SELECT * FROM movies;", con).head(10).transpose()

,0,1,2,3,4,5,6,7,8,9
idx,0,0,0,0,0,0,0,0,0,0
id,,,2,3,,5,6,,8,9
imdb_id,,,tt0094675,tt0092149,,tt0113101,tt0107286,,tt0825671,tt0425473
original_language,,,fi,fi,,en,en,,en,de
original_title,,,Ariel,Varjoja paratiisissa,,Four Rooms,Judgment Night,,Life in Loops (A Megacities RMX),Sonntag im August
popularity,,,4.481,4.016,,8.565,6.617,,0.6,1.052
release_date,,,1988-10-21,1986-10-17,,1995-12-09,1993-10-15,,2006-01-01,2004-09-02
revenue,,,0,0,,4.3e+06,12,,0,0
runtime,,,73,72,,98,110,,80,15
status,,,Released,Released,,Released,Released,,Released,Released


Number of missing ids due to quality of data source:

In [40]:
pd.read_sql_query("SELECT COUNT(imdb_id) FROM movies WHERE imdb_id = '';", con)

,COUNT(imdb_id)
0,92866


Count of doubled entries due to streaming the data from API for ETL process being under continues design:

In [41]:
pd.read_sql_query("SELECT imdb_id, COUNT(imdb_id) AS imdb_id_count FROM movies WHERE imdb_id <> ('') GROUP BY imdb_id ORDER BY imdb_id_count DESC;", con).head()

,imdb_id,imdb_id_count
0,tt2226647,3
1,tt2618150,3
2,tt0002083,2
3,tt0021889,2
4,tt0023229,2


Cleaning Movies table with a use of simple filtering:

In [42]:
pd.read_sql_query("SELECT DISTINCT * FROM movies WHERE imdb_id <> ''", con).head(10).transpose()

,0,1,2,3,4,5,6,7,8,9
idx,0,0,0,0,0,0,0,0,0,0
id,2,3,5,6,8,9,11,12,13,14
imdb_id,tt0094675,tt0092149,tt0113101,tt0107286,tt0825671,tt0425473,tt0076759,tt0266543,tt0109830,tt0169547
original_language,fi,fi,en,en,en,de,en,en,en,en
original_title,Ariel,Varjoja paratiisissa,Four Rooms,Judgment Night,Life in Loops (A Megacities RMX),Sonntag im August,Star Wars,Finding Nemo,Forrest Gump,American Beauty
popularity,4.481,4.016,8.565,6.617,0.6,1.052,36.399,21.046,20.479,13.725
release_date,1988-10-21,1986-10-17,1995-12-09,1993-10-15,2006-01-01,2004-09-02,1977-05-25,2003-05-30,1994-07-06,1999-09-15
revenue,0,0,4.3e+06,12,0,0,7.75398e+08,9.40336e+08,6.77945e+08,3.56297e+08
runtime,73,72,98,110,80,15,121,100,142,122
status,Released,Released,Released,Released,Released,Released,Released,Released,Released,Released


Checking the results of cleaning:

In [43]:
pd.read_sql_query("SELECT imdb_id, COUNT(imdb_id) AS imdb_id_count FROM (SELECT DISTINCT * FROM movies WHERE imdb_id <> '') GROUP BY imdb_id ORDER BY imdb_id_count DESC", con).head()

,imdb_id,imdb_id_count
0,tt1684934,2
1,tt0000001,1
2,tt0000002,1
3,tt0000003,1
4,tt0000004,1


Is imdb_id_count == 'tt1684934' realy a duplicate entry?

In [44]:
pd.read_sql_query("SELECT DISTINCT * FROM movies WHERE imdb_id IN ('tt1684934')", con).head().transpose()

,0,1
idx,0,0
id,56468,138076
imdb_id,tt1684934,tt1684934
original_language,en,en
original_title,Video Games Live,Video Games Live: Level 2
popularity,0.6,0.6
release_date,2010-06-02,2010-06-07
revenue,0,0
runtime,60,100
status,Released,Released


## Basic characteristics of Movie table after cleaning

Tha shape of Movies table:

In [45]:
pd.read_sql_query("SELECT DISTINCT * FROM movies WHERE imdb_id <> ''", con).shape

(105551, 13)

Attributes of fields:

In [46]:
pd.read_sql_query("SELECT DISTINCT * FROM movies WHERE imdb_id <> ''", con).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105551 entries, 0 to 105550
Data columns (total 13 columns):
idx                  105551 non-null int64
id                   105551 non-null int64
imdb_id              105551 non-null object
original_language    105551 non-null object
original_title       105551 non-null object
popularity           105551 non-null float64
release_date         105551 non-null object
revenue              105551 non-null float64
runtime              105551 non-null object
status               105551 non-null object
title                105551 non-null object
vote_average         105551 non-null float64
vote_count           105551 non-null int64
dtypes: float64(3), int64(3), object(7)
memory usage: 10.5+ MB


Transposing Movies table:

In [47]:
pd.read_sql_query("SELECT DISTINCT * FROM movies WHERE imdb_id <> ''", con).head(100).transpose()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
idx,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id,2,3,5,6,8,9,11,12,13,14,...,123,124,125,126,127,128,129,132,133,134
imdb_id,tt0094675,tt0092149,tt0113101,tt0107286,tt0825671,tt0425473,tt0076759,tt0266543,tt0109830,tt0169547,...,tt0077869,tt0086961,tt0080668,tt0082631,tt0084549,tt0119698,tt0245429,tt0065780,tt0054205,tt0190590
original_language,fi,fi,en,en,en,de,en,en,en,en,...,en,pl,pl,pl,pl,ja,ja,en,en,en
original_title,Ariel,Varjoja paratiisissa,Four Rooms,Judgment Night,Life in Loops (A Megacities RMX),Sonntag im August,Star Wars,Finding Nemo,Forrest Gump,American Beauty,...,The Lord of the Rings,Bez końca,Dworzec,Krótki dzień pracy,Przypadek,もののけ姫,千と千尋の神隠し,Gimme Shelter,Primary,"O Brother, Where Art Thou?"
popularity,4.481,4.016,8.565,6.617,0.6,1.052,36.399,21.046,20.479,13.725,...,8.42,1.131,1.12,0.6,3.225,14.525,15.445,2.977,0.6,9.963
release_date,1988-10-21,1986-10-17,1995-12-09,1993-10-15,2006-01-01,2004-09-02,1977-05-25,2003-05-30,1994-07-06,1999-09-15,...,1978-11-15,1985-06-17,1980-01-01,1981-01-01,1987-01-10,1997-07-12,2001-07-20,1970-12-06,1960-01-01,2000-08-30
revenue,0,0,4.3e+06,12,0,0,7.75398e+08,9.40336e+08,6.77945e+08,3.56297e+08,...,3.04714e+07,0,0,0,0,1.59375e+08,2.74925e+08,252570,0,7.18683e+07
runtime,73,72,98,110,80,15,121,100,142,122,...,132,108,13,73,123,134,125,90,60,106
status,Released,Released,Released,Released,Released,Released,Released,Released,Released,Released,...,Released,Released,Released,Released,Released,Released,Released,Released,Released,Released


# Genres Dataset

## Basic characteristics of Genres table before cleaning:
Tha shape of Genres table:

In [48]:
pd.read_sql_query("SELECT * FROM genres;", con).shape

(45004, 4)

Attributes of fields:

In [49]:
pd.read_sql_query("SELECT * FROM genres;", con).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45004 entries, 0 to 45003
Data columns (total 4 columns):
idx        45004 non-null int64
id         45004 non-null object
name       45004 non-null object
imdb_id    45004 non-null object
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


First five rows of Genres table:

In [50]:
pd.read_sql_query("SELECT * FROM genres;", con).head(10).transpose()

,0,1,2,3,4,5,6,7,8,9
idx,0,0,0,0,0,0,0,0,0,0
id,18,18,28,18,16,35,18,9648,18,27
name,Drama,Drama,Action,Drama,Animation,Comedy,Drama,Mystery,Drama,Horror
imdb_id,tt0094675,tt0092149,tt0107286,tt0425473,tt0266543,tt0109830,tt0169547,tt0033467,tt0168629,tt0411267


Number of missing ids due to quality of data source:

In [51]:
pd.read_sql_query("SELECT COUNT(imdb_id) AS imdb_id_count FROM genres WHERE imdb_id = '';", con)

,imdb_id_count
0,6828


Count of doubled entries due to streaming the data from API for ETL process being under continues design:

In [52]:
pd.read_sql_query("SELECT imdb_id, COUNT(imdb_id) AS imdb_id_count FROM genres WHERE imdb_id <> ('') GROUP BY imdb_id ORDER BY imdb_id_count DESC;", con).head()

,imdb_id,imdb_id_count
0,tt0022905,2
1,tt0023074,2
2,tt0023470,2
3,tt0024772,2
4,tt0024963,2


Cleaning Genres table with a use of simple filtering:

In [53]:
pd.read_sql_query("SELECT DISTINCT * FROM genres WHERE imdb_id <> ''", con).head(10).transpose()

,0,1,2,3,4,5,6,7,8,9
idx,0,0,0,0,0,0,0,0,0,0
id,18,18,28,18,16,35,18,9648,18,27
name,Drama,Drama,Action,Drama,Animation,Comedy,Drama,Mystery,Drama,Horror
imdb_id,tt0094675,tt0092149,tt0107286,tt0425473,tt0266543,tt0109830,tt0169547,tt0033467,tt0168629,tt0411267


Checking the results of cleaning:

In [54]:
pd.read_sql_query("SELECT imdb_id, COUNT(imdb_id) as imdb_id_count FROM (SELECT DISTINCT * FROM genres WHERE imdb_id <> '') GROUP BY imdb_id ORDER BY imdb_id_count DESC;", con).head()

,imdb_id,imdb_id_count
0,tt0000001,1
1,tt0000002,1
2,tt0000010,1
3,tt0000012,1
4,tt0000246,1


## Basic characteristics of Genres table after cleaning

Tha shape of Movies table:

In [55]:
pd.read_sql_query("SELECT DISTINCT * FROM genres WHERE imdb_id <> '';", con).shape

(38124, 4)

Attributes of fields:

In [56]:
pd.read_sql_query("SELECT DISTINCT * FROM genres WHERE imdb_id <> '';", con).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38124 entries, 0 to 38123
Data columns (total 4 columns):
idx        38124 non-null int64
id         38124 non-null int64
name       38124 non-null object
imdb_id    38124 non-null object
dtypes: int64(2), object(2)
memory usage: 1.2+ MB


Transposing Movies table:

In [57]:
pd.read_sql_query("SELECT DISTINCT * FROM genres WHERE imdb_id <> '';", con).head(10).transpose()

,0,1,2,3,4,5,6,7,8,9
idx,0,0,0,0,0,0,0,0,0,0
id,18,18,28,18,16,35,18,9648,18,27
name,Drama,Drama,Action,Drama,Animation,Comedy,Drama,Mystery,Drama,Horror
imdb_id,tt0094675,tt0092149,tt0107286,tt0425473,tt0266543,tt0109830,tt0169547,tt0033467,tt0168629,tt0411267


# Production Countries Dataset

## Basic characteristics of Production Countries table before cleaning:
Tha shape of Production Countries table:

In [58]:
pd.read_sql_query("SELECT * FROM productioncountries;", con).shape

(93440, 3)

Attributes of fields:

In [59]:
pd.read_sql_query("SELECT * FROM productioncountries;", con).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93440 entries, 0 to 93439
Data columns (total 3 columns):
iso_3166_1    93440 non-null object
name          93440 non-null object
imdb_id       93440 non-null object
dtypes: object(3)
memory usage: 2.1+ MB


First five rows of Production Countries table:

In [60]:
pd.read_sql_query("SELECT * FROM productioncountries;", con).head(10).transpose()

,0,1,2,3,4,5,6,7,8,9
iso_3166_1,FI,FI,JP,DE,US,US,US,US,AR,DE
name,Finland,Finland,Japan,Germany,United States of America,United States of America,United States of America,United States of America,Argentina,Germany
imdb_id,tt0094675,tt0092149,tt0107286,tt0425473,tt0266543,tt0109830,tt0169547,tt0033467,tt0168629,tt0411267


Number of missing ids due to quality of data source:

In [61]:
pd.read_sql_query("SELECT COUNT(imdb_id) as imdb_id_count FROM productioncountries WHERE imdb_id IN ('');", con).head()

,imdb_id_count
0,31450


Count of doubled entries due to streaming the data from API for ETL process being under continues design:

In [62]:
pd.read_sql_query("SELECT imdb_id, COUNT(imdb_id) AS imdb_id_count FROM productioncountries WHERE imdb_id <> ('') GROUP BY imdb_id ORDER BY imdb_id_count DESC;", con).head()

,imdb_id,imdb_id_count
0,tt1947964,3
1,tt0058137,2
2,tt0087419,2
3,tt0096396,2
4,tt0098240,2


Cleaning Production Countries table with a use of simple filtering:

In [63]:
pd.read_sql_query("SELECT DISTINCT * FROM productioncountries WHERE imdb_id <> ''", con).head(10).transpose()

,0,1,2,3,4,5,6,7,8,9
iso_3166_1,FI,FI,JP,DE,US,US,US,US,AR,DE
name,Finland,Finland,Japan,Germany,United States of America,United States of America,United States of America,United States of America,Argentina,Germany
imdb_id,tt0094675,tt0092149,tt0107286,tt0425473,tt0266543,tt0109830,tt0169547,tt0033467,tt0168629,tt0411267


Checking the results of cleaning:

In [64]:
pd.read_sql_query("SELECT imdb_id, COUNT(imdb_id) as imdb_id_count FROM (SELECT DISTINCT * FROM productioncountries WHERE imdb_id <> '') GROUP BY imdb_id ORDER BY imdb_id_count DESC;", con).head()

,imdb_id,imdb_id_count
0,tt0000001,1
1,tt0000002,1
2,tt0000003,1
3,tt0000007,1
4,tt0000008,1


## Basic characteristics of Production Countries table after cleaning

The shape of Production Countries table:

In [65]:
pd.read_sql_query("SELECT DISTINCT * FROM productioncountries WHERE imdb_id <> ''", con).shape

(61972, 3)

Attributes of fields:

In [66]:
pd.read_sql_query("SELECT DISTINCT * FROM productioncountries WHERE imdb_id <> ''", con).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61972 entries, 0 to 61971
Data columns (total 3 columns):
iso_3166_1    61972 non-null object
name          61972 non-null object
imdb_id       61972 non-null object
dtypes: object(3)
memory usage: 1.4+ MB


Transposing Production Countriestable:

In [67]:
pd.read_sql_query("SELECT DISTINCT * FROM productioncountries WHERE imdb_id <> ''", con).head(100).transpose()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
iso_3166_1,FI,FI,JP,DE,US,US,US,US,AR,DE,...,GB,US,US,US,DK,ES,US,CA,US,DE
name,Finland,Finland,Japan,Germany,United States of America,United States of America,United States of America,United States of America,Argentina,Germany,...,United Kingdom,United States of America,United States of America,United States of America,Denmark,Spain,United States of America,Canada,United States of America,Germany
imdb_id,tt0094675,tt0092149,tt0107286,tt0425473,tt0266543,tt0109830,tt0169547,tt0033467,tt0168629,tt0411267,...,tt0061589,tt0022913,tt0107048,tt0021814,tt0164756,tt0275491,tt0246578,tt0388795,tt0020629,tt0093191


# Cast Dataset

## Basic characteristics of Cast table before cleaning:
Tha shape of Cast table:

In [68]:
pd.read_sql_query("SELECT * FROM cast", con).shape

(314350, 10)

Attributes of fields:

In [69]:
pd.read_sql_query("SELECT * FROM cast", con).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314350 entries, 0 to 314349
Data columns (total 10 columns):
idx             314350 non-null int64
cast_id         314350 non-null int64
character       314350 non-null object
credit_id       314350 non-null object
gender          314350 non-null int64
id              314350 non-null int64
name            314350 non-null object
Movie.order     314350 non-null int64
profile_path    314350 non-null object
Movie.id        314350 non-null int64
dtypes: int64(6), object(4)
memory usage: 24.0+ MB


First five rows of Cast table:

In [70]:
pd.read_sql_query("SELECT * FROM cast;", con).head(10).transpose()

,0,1,2,3,4,5,6,7,8,9
idx,0,1,2,3,0,1,2,3,4,5
cast_id,3,4,5,6,5,6,7,9,10,11
character,Taisto Olavi Kasurinen,Irmeli Katariina Pihlaja,Mikkonen,Riku,Nikander,Ilona Rajamäki,Melartin,Co-worker,Ilona's Girlfriend,Shop Steward
credit_id,52fe420dc3a36847f8000029,52fe420dc3a36847f800002d,52fe420dc3a36847f8000031,52fe420dc3a36847f8000035,52fe420dc3a36847f8000087,52fe420dc3a36847f800008b,52fe420dc3a36847f800008f,52fe420dc3a36847f8000099,52fe420dc3a36847f800009d,52fe420dc3a36847f80000a1
gender,2,1,2,2,2,1,2,2,1,2
id,54768,54769,4826,54770,4826,5999,4828,53508,1086499,222320
name,Turo Pajala,Susanna Haavisto,Matti Pellonpää,Eetu Hilkamo,Matti Pellonpää,Kati Outinen,Sakari Kuosmanen,Esko Nikkari,Kylli Köngäs,Pekka Laiho
Movie.order,0,1,2,3,0,1,2,3,4,5
profile_path,,,/7WuLvkuWphUAtW6QQwtF3WrwUKE.jpg,,/7WuLvkuWphUAtW6QQwtF3WrwUKE.jpg,/cjcuhAXRcg8C0VIDHtfz1BIVE3m.jpg,/fziy9WDgG5aW2fiKpK3kK0lfhuS.jpg,/mNiyD3BOzWba4CB0VTbIqA4EAVm.jpg,,/oLRsjrS3J9GMrPz6RsXCxhOJo9g.jpg
Movie.id,2,2,2,2,3,3,3,3,3,3


Number of missing ids due to quality of data source:

In [71]:
pd.read_sql_query("SELECT COUNT('Movie.Id') as movie_id_count FROM cast WHERE 'Movie.id' ISNULL", con).head()

,movie_id_count
0,0


Cleaning Cast table with a use of simple filtering:

In [72]:
pd.read_sql_query("SELECT DISTINCT * FROM cast WHERE 'Movie.id' NOT NULL", con).head(10).transpose()

,0,1,2,3,4,5,6,7,8,9
idx,0,1,2,3,0,1,2,3,4,5
cast_id,3,4,5,6,5,6,7,9,10,11
character,Taisto Olavi Kasurinen,Irmeli Katariina Pihlaja,Mikkonen,Riku,Nikander,Ilona Rajamäki,Melartin,Co-worker,Ilona's Girlfriend,Shop Steward
credit_id,52fe420dc3a36847f8000029,52fe420dc3a36847f800002d,52fe420dc3a36847f8000031,52fe420dc3a36847f8000035,52fe420dc3a36847f8000087,52fe420dc3a36847f800008b,52fe420dc3a36847f800008f,52fe420dc3a36847f8000099,52fe420dc3a36847f800009d,52fe420dc3a36847f80000a1
gender,2,1,2,2,2,1,2,2,1,2
id,54768,54769,4826,54770,4826,5999,4828,53508,1086499,222320
name,Turo Pajala,Susanna Haavisto,Matti Pellonpää,Eetu Hilkamo,Matti Pellonpää,Kati Outinen,Sakari Kuosmanen,Esko Nikkari,Kylli Köngäs,Pekka Laiho
Movie.order,0,1,2,3,0,1,2,3,4,5
profile_path,,,/7WuLvkuWphUAtW6QQwtF3WrwUKE.jpg,,/7WuLvkuWphUAtW6QQwtF3WrwUKE.jpg,/cjcuhAXRcg8C0VIDHtfz1BIVE3m.jpg,/fziy9WDgG5aW2fiKpK3kK0lfhuS.jpg,/mNiyD3BOzWba4CB0VTbIqA4EAVm.jpg,,/oLRsjrS3J9GMrPz6RsXCxhOJo9g.jpg
Movie.id,2,2,2,2,3,3,3,3,3,3


## Basic characteristics of Cast table after cleaning

Tha shape of Cast table:

In [73]:
pd.read_sql_query("SELECT DISTINCT * FROM cast WHERE 'Movie.id' NOT NULL", con).shape

(314079, 10)

Attributes of fields:

In [74]:
pd.read_sql_query("SELECT DISTINCT * FROM cast WHERE 'Movie.id' NOT NULL", con).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314079 entries, 0 to 314078
Data columns (total 10 columns):
idx             314079 non-null int64
cast_id         314079 non-null int64
character       314079 non-null object
credit_id       314079 non-null object
gender          314079 non-null int64
id              314079 non-null int64
name            314079 non-null object
Movie.order     314079 non-null int64
profile_path    314079 non-null object
Movie.id        314079 non-null int64
dtypes: int64(6), object(4)
memory usage: 24.0+ MB


Transposing Cast table:

In [75]:
pd.read_sql_query("SELECT DISTINCT * FROM cast WHERE 'Movie.id' NOT NULL", con).head(100).transpose()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
idx,0,1,2,3,0,1,2,3,4,5,...,38,39,40,41,42,43,44,45,46,47
cast_id,3,4,5,6,5,6,7,9,10,11,...,47,48,49,50,51,52,53,54,55,56
character,Taisto Olavi Kasurinen,Irmeli Katariina Pihlaja,Mikkonen,Riku,Nikander,Ilona Rajamäki,Melartin,Co-worker,Ilona's Girlfriend,Shop Steward,...,Stormtrooper (uncredited),Wioslea (uncredited),Dr. Evazan (uncredited),Cantina Patron (uncredited),Greedo (uncredited),Hrchek Kal Fas (uncredited),Jawa (uncredited),Stormtrooper / Tusken Raider / Death Star Troo...,Jawa (uncredited),Garindan (uncredited)
credit_id,52fe420dc3a36847f8000029,52fe420dc3a36847f800002d,52fe420dc3a36847f8000031,52fe420dc3a36847f8000035,52fe420dc3a36847f8000087,52fe420dc3a36847f800008b,52fe420dc3a36847f800008f,52fe420dc3a36847f8000099,52fe420dc3a36847f800009d,52fe420dc3a36847f80000a1,...,52fe420ec3a36847f80004fb,52fe420ec3a36847f80004ff,52fe420ec3a36847f8000503,52fe420ec3a36847f8000507,52fe420ec3a36847f800050b,52fe420ec3a36847f800050f,52fe420ec3a36847f8000513,52fe420ec3a36847f8000517,52fe420ec3a36847f800051b,52fe420ec3a36847f800051f
gender,2,1,2,2,2,1,2,2,1,2,...,2,0,2,0,1,0,0,2,0,0
id,54768,54769,4826,54770,4826,5999,4828,53508,1086499,222320,...,1178140,186229,1271066,1271067,104149,1271068,1271069,53587,1271070,1271071
name,Turo Pajala,Susanna Haavisto,Matti Pellonpää,Eetu Hilkamo,Matti Pellonpää,Kati Outinen,Sakari Kuosmanen,Esko Nikkari,Kylli Köngäs,Pekka Laiho,...,Tim Condren,Barry Copping,Alfie Curtis,Robert Davies,Maria De Aragon,Robert A. Denham,Frazer Diamond,Peter Diamond,Warwick Diamond,Sadie Eden
Movie.order,0,1,2,3,0,1,2,3,4,5,...,38,39,40,41,42,43,44,45,46,47
profile_path,,,/7WuLvkuWphUAtW6QQwtF3WrwUKE.jpg,,/7WuLvkuWphUAtW6QQwtF3WrwUKE.jpg,/cjcuhAXRcg8C0VIDHtfz1BIVE3m.jpg,/fziy9WDgG5aW2fiKpK3kK0lfhuS.jpg,/mNiyD3BOzWba4CB0VTbIqA4EAVm.jpg,,/oLRsjrS3J9GMrPz6RsXCxhOJo9g.jpg,...,/b2LuPeiNkiN7YT0mpJ1O0C2BV58.jpg,,/5jKHKbIF1cEWeG2sPAzHScgGW7n.jpg,,/rnaslrjV5ui6cKphksSni3K0TVQ.jpg,,/9e3i9TJ0pp5zbUnDljNuvkjhgCW.jpg,/rIf04LU2CsdzdvUJghFVVjdWcm6.jpg,/3E5Ktz0o6k4Xz6iTq2zCVHsxleX.jpg,
Movie.id,2,2,2,2,3,3,3,3,3,3,...,11,11,11,11,11,11,11,11,11,11


# Crew Dataset

## Basic characteristics of Crew table before cleaning:
Tha shape of Crew table:

In [76]:
pd.read_sql_query("SELECT * FROM crew", con).shape

(312963, 9)

Attributes of fields:

In [77]:
pd.read_sql_query("SELECT * FROM crew", con).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312963 entries, 0 to 312962
Data columns (total 9 columns):
idx             312963 non-null object
credit_id       312963 non-null object
department      312963 non-null object
gender          312963 non-null object
id              312963 non-null object
job             312963 non-null object
name            312963 non-null object
profile_path    312963 non-null object
Movie.id        312963 non-null object
dtypes: object(9)
memory usage: 21.5+ MB


First five rows of Crew table:

In [78]:
pd.read_sql_query("SELECT * FROM crew;", con).head(10).transpose()

,0,1,2,3,4,5,6,7,8,9
idx,0,1,2,3,4,5,6,0,1,2
credit_id,52fe420dc3a36847f800001f,52fe420dc3a36847f8000025,52fe420dc3a36847f800003b,52fe420dc3a36847f8000041,52fe420dc3a36847f8000047,52fe420dc3a36847f800004d,5be040dac3a3682b34043d42,52fe420dc3a36847f8000071,52fe420dc3a36847f8000077,52fe420dc3a36847f8000083
department,Writing,Directing,Camera,Editing,Art,Costume & Make-Up,Production,Directing,Writing,Editing
gender,2,2,2,1,2,1,2,2,2,1
id,16767,16767,16769,54766,53836,54771,16767,16767,16767,54766
job,Screenplay,Director,Director of Photography,Editor,Production Design,Costume Design,Producer,Director,Screenplay,Editor
name,Aki Kaurismäki,Aki Kaurismäki,Timo Salminen,Raija Talvio,Risto Karhula,Tuula Hilkamo,Aki Kaurismäki,Aki Kaurismäki,Aki Kaurismäki,Raija Talvio
profile_path,/rwwpC3H1rwLvSv7pIzAeII2aA2k.jpg,/rwwpC3H1rwLvSv7pIzAeII2aA2k.jpg,,,,,/rwwpC3H1rwLvSv7pIzAeII2aA2k.jpg,/rwwpC3H1rwLvSv7pIzAeII2aA2k.jpg,/rwwpC3H1rwLvSv7pIzAeII2aA2k.jpg,
Movie.id,2,2,2,2,2,2,2,3,3,3


Number of missing ids due to quality of data source:

In [79]:
pd.read_sql_query("SELECT COUNT('Movie.Id') as movie_id_count FROM crew WHERE 'Movie.id' ISNULL", con).head()

,movie_id_count
0,0


Cleaning Crew table with a use of simple filtering:

In [80]:
pd.read_sql_query("SELECT DISTINCT * FROM crew WHERE 'Movie.id' NOT NULL", con).head(10).transpose()

,0,1,2,3,4,5,6,7,8,9
idx,0,1,2,3,4,5,6,0,1,2
credit_id,52fe420dc3a36847f800001f,52fe420dc3a36847f8000025,52fe420dc3a36847f800003b,52fe420dc3a36847f8000041,52fe420dc3a36847f8000047,52fe420dc3a36847f800004d,5be040dac3a3682b34043d42,52fe420dc3a36847f8000071,52fe420dc3a36847f8000077,52fe420dc3a36847f8000083
department,Writing,Directing,Camera,Editing,Art,Costume & Make-Up,Production,Directing,Writing,Editing
gender,2,2,2,1,2,1,2,2,2,1
id,16767,16767,16769,54766,53836,54771,16767,16767,16767,54766
job,Screenplay,Director,Director of Photography,Editor,Production Design,Costume Design,Producer,Director,Screenplay,Editor
name,Aki Kaurismäki,Aki Kaurismäki,Timo Salminen,Raija Talvio,Risto Karhula,Tuula Hilkamo,Aki Kaurismäki,Aki Kaurismäki,Aki Kaurismäki,Raija Talvio
profile_path,/rwwpC3H1rwLvSv7pIzAeII2aA2k.jpg,/rwwpC3H1rwLvSv7pIzAeII2aA2k.jpg,,,,,/rwwpC3H1rwLvSv7pIzAeII2aA2k.jpg,/rwwpC3H1rwLvSv7pIzAeII2aA2k.jpg,/rwwpC3H1rwLvSv7pIzAeII2aA2k.jpg,
Movie.id,2,2,2,2,2,2,2,3,3,3


## Basic characteristics of Crew table after cleaning

Tha shape of Cast table:

In [81]:
pd.read_sql_query("SELECT DISTINCT * FROM crew WHERE 'Movie.id' NOT NULL", con).shape

(304561, 9)

Attributes of fields:

In [82]:
pd.read_sql_query("SELECT DISTINCT * FROM crew WHERE 'Movie.id' NOT NULL", con).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304561 entries, 0 to 304560
Data columns (total 9 columns):
idx             304561 non-null object
credit_id       304561 non-null object
department      304561 non-null object
gender          304561 non-null object
id              304561 non-null object
job             304561 non-null object
name            304561 non-null object
profile_path    304561 non-null object
Movie.id        304561 non-null object
dtypes: object(9)
memory usage: 20.9+ MB


Transposing Crew table:

In [83]:
pd.read_sql_query("SELECT DISTINCT * FROM crew WHERE 'Movie.id' NOT NULL", con).head(10).transpose()

,0,1,2,3,4,5,6,7,8,9
idx,0,1,2,3,4,5,6,0,1,2
credit_id,52fe420dc3a36847f800001f,52fe420dc3a36847f8000025,52fe420dc3a36847f800003b,52fe420dc3a36847f8000041,52fe420dc3a36847f8000047,52fe420dc3a36847f800004d,5be040dac3a3682b34043d42,52fe420dc3a36847f8000071,52fe420dc3a36847f8000077,52fe420dc3a36847f8000083
department,Writing,Directing,Camera,Editing,Art,Costume & Make-Up,Production,Directing,Writing,Editing
gender,2,2,2,1,2,1,2,2,2,1
id,16767,16767,16769,54766,53836,54771,16767,16767,16767,54766
job,Screenplay,Director,Director of Photography,Editor,Production Design,Costume Design,Producer,Director,Screenplay,Editor
name,Aki Kaurismäki,Aki Kaurismäki,Timo Salminen,Raija Talvio,Risto Karhula,Tuula Hilkamo,Aki Kaurismäki,Aki Kaurismäki,Aki Kaurismäki,Raija Talvio
profile_path,/rwwpC3H1rwLvSv7pIzAeII2aA2k.jpg,/rwwpC3H1rwLvSv7pIzAeII2aA2k.jpg,,,,,/rwwpC3H1rwLvSv7pIzAeII2aA2k.jpg,/rwwpC3H1rwLvSv7pIzAeII2aA2k.jpg,/rwwpC3H1rwLvSv7pIzAeII2aA2k.jpg,
Movie.id,2,2,2,2,2,2,2,3,3,3


# Overview Dataset

## Basic characteristics of Overview table before cleaning:
Tha shape of Overview table:

In [84]:
pd.read_sql_query("SELECT * FROM overview", con).shape

(25154, 4)

Attributes of fields:

In [85]:
pd.read_sql_query("SELECT DISTINCT * FROM overview WHERE 'Movie.id' NOT NULL", con).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15001 entries, 0 to 15000
Data columns (total 4 columns):
idx         15001 non-null int64
id          15001 non-null object
imdb_id     15001 non-null object
overview    15001 non-null object
dtypes: int64(1), object(3)
memory usage: 468.9+ KB


First five rows of Overview table:

In [86]:
pd.read_sql_query("SELECT * FROM overview", con).head(15).transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
idx,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
id,,2,3,,5,6,,8,9,,11,12,13,14,15
imdb_id,,tt0094675,tt0092149,,tt0113101,tt0107286,,tt0825671,tt0425473,,tt0076759,tt0266543,tt0109830,tt0169547,tt0033467
overview,,Taisto Kasurinen is a Finnish coal miner whose...,"An episode in the life of Nikander, a garbage ...",,It's Ted the Bellhop's first night on the job....,"While racing to a boxing match, Frank, Mike, J...",,Timo Novotny labels his new project an experim...,,,Princess Leia is captured and held hostage by ...,"Nemo, an adventurous young clownfish, is unexp...",A man with a low IQ has accomplished great thi...,"Lester Burnham, a depressed suburban father in...","Newspaper magnate, Charles Foster Kane is take..."


Number of missing ids due to quality of data source:

In [87]:
pd.read_sql_query("SELECT COUNT(imdb_id) as imdb_id_count FROM overview WHERE imdb_id IN ('')", con).head()

,imdb_id_count
0,10545


Count of doubled entries due to streaming the data from API for ETL process being under continue design:

In [88]:
pd.read_sql_query("SELECT imdb_id, COUNT(imdb_id) AS imdb_id_count FROM overview WHERE imdb_id <> ('') GROUP BY imdb_id ORDER BY imdb_id_count DESC;", con).head()

,imdb_id,imdb_id_count
0,tt0461994,3
1,tt0076489,2
2,tt0094597,2
3,tt0106273,2
4,tt0112462,2


Cleaning Overview table with a use of simple filtering:

In [89]:
pd.read_sql_query("SELECT DISTINCT * FROM overview WHERE imdb_id <> ('') AND overview <> ('')", con).head(15).transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
idx,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
id,2,3,5,6,8,11,12,13,14,15,16,17,18,19,20
imdb_id,tt0094675,tt0092149,tt0113101,tt0107286,tt0825671,tt0076759,tt0266543,tt0109830,tt0169547,tt0033467,tt0168629,tt0411267,tt0119116,tt0017136,tt0314412
overview,Taisto Kasurinen is a Finnish coal miner whose...,"An episode in the life of Nikander, a garbage ...",It's Ted the Bellhop's first night on the job....,"While racing to a boxing match, Frank, Mike, J...",Timo Novotny labels his new project an experim...,Princess Leia is captured and held hostage by ...,"Nemo, an adventurous young clownfish, is unexp...",A man with a low IQ has accomplished great thi...,"Lester Burnham, a depressed suburban father in...","Newspaper magnate, Charles Foster Kane is take...","Selma, a Czech immigrant on the verge of blind...",Adèle and her daughter Sarah are traveling on ...,"In 2257, a taxi driver is unintentionally give...",In a futuristic city sharply divided between t...,A Pedro Almodovar production in which a fatall...


## Basic characteristics of Overview table after cleaning

Tha shape of Overview table:

In [90]:
pd.read_sql_query("SELECT DISTINCT * FROM overview WHERE imdb_id <> ('') AND overview <> ('')", con).shape

(14381, 4)

Attributes of fields:

In [91]:
pd.read_sql_query("SELECT DISTINCT * FROM overview WHERE imdb_id <> ('') AND overview <> ('')", con).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14381 entries, 0 to 14380
Data columns (total 4 columns):
idx         14381 non-null int64
id          14381 non-null int64
imdb_id     14381 non-null object
overview    14381 non-null object
dtypes: int64(2), object(2)
memory usage: 449.5+ KB


Transposing Overview table:

In [92]:
pd.read_sql_query("SELECT DISTINCT * FROM overview WHERE imdb_id <> ('')", con).head(100).transpose()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
idx,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id,2,3,5,6,8,9,11,12,13,14,...,123,124,125,126,127,128,129,132,133,134
imdb_id,tt0094675,tt0092149,tt0113101,tt0107286,tt0825671,tt0425473,tt0076759,tt0266543,tt0109830,tt0169547,...,tt0077869,tt0086961,tt0080668,tt0082631,tt0084549,tt0119698,tt0245429,tt0065780,tt0054205,tt0190590
overview,Taisto Kasurinen is a Finnish coal miner whose...,"An episode in the life of Nikander, a garbage ...",It's Ted the Bellhop's first night on the job....,"While racing to a boxing match, Frank, Mike, J...",Timo Novotny labels his new project an experim...,,Princess Leia is captured and held hostage by ...,"Nemo, an adventurous young clownfish, is unexp...",A man with a low IQ has accomplished great thi...,"Lester Burnham, a depressed suburban father in...",...,The Fellowship of the Ring embark on a journey...,Taking place in a 1982 Poland a translator los...,Railway Station is a documentary about a Warsa...,"Kieślowski’s study of a political strike, cont...",Witek runs after a train. Three variations fol...,"Ashitaka, a prince of the disappearing Ainu tr...","A young girl, Chihiro, becomes trapped in a st...",The landmark documentary about the tragically ...,Primary is a documentary film about the primar...,"In the deep south during the 1930s, three esca..."


# Keywords Dataset

## Basic characteristics of Keywords table before cleaning:
Tha shape of Overview table:

In [94]:
pd.read_sql_query("SELECT * FROM keywords", con).shape

(55206, 3)

Attributes of fields:

In [95]:
pd.read_sql_query("SELECT DISTINCT * FROM keywords WHERE 'Movie.id' NOT NULL", con).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55064 entries, 0 to 55063
Data columns (total 3 columns):
idx         55064 non-null int64
keywords    55064 non-null object
id          55064 non-null int64
dtypes: int64(2), object(1)
memory usage: 1.3+ MB


First five rows of Keywords table:

In [96]:
pd.read_sql_query("SELECT * FROM keywords", con).head(15).transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
idx,1361,1787,3700,10183,612,613,616,622,922,2700,9706,12670,160488,187056,520
keywords,salesclerk,helsinki,garbage,independent film,hotel,new year's eve,witch,bet,hotel room,sperm,anthology,los angeles,hoodlum,woman director,chicago
id,3,3,3,3,5,5,5,5,5,5,5,5,5,5,6


Number of missing ids due to quality of data source:

In [97]:
pd.read_sql_query("SELECT COUNT(id) as id_count FROM keywords WHERE id IN ('')", con).head()

,id_count
0,0


Cleaning Keywords table with a use of simple filtering:

In [103]:
pd.read_sql_query("SELECT DISTINCT * FROM keywords WHERE id <> ('') AND keywords <> ('')", con).head(15).transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
idx,1361,1787,3700,10183,612,613,616,622,922,2700,9706,12670,160488,187056,520
keywords,salesclerk,helsinki,garbage,independent film,hotel,new year's eve,witch,bet,hotel room,sperm,anthology,los angeles,hoodlum,woman director,chicago
id,3,3,3,3,5,5,5,5,5,5,5,5,5,5,6


## Basic characteristics of Keywors table after cleaning

Tha shape of Keywords table:

In [104]:
pd.read_sql_query("SELECT DISTINCT * FROM keywords WHERE id <> ('') AND keywords <> ('')", con).shape

(55064, 3)

Attributes of fields:

In [107]:
pd.read_sql_query("SELECT DISTINCT * FROM keywords WHERE id <> ('') AND keywords <> ('')", con).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55064 entries, 0 to 55063
Data columns (total 3 columns):
idx         55064 non-null int64
keywords    55064 non-null object
id          55064 non-null int64
dtypes: int64(2), object(1)
memory usage: 1.3+ MB


Transposing Keywords table:

In [108]:
pd.read_sql_query("SELECT DISTINCT * FROM keywords WHERE id <> ('') AND keywords <> ('');", con).head(100).transpose()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
idx,1361,1787,3700,10183,612,613,616,622,922,2700,...,224279,229238,229299,239239,736,918,1196,2110,3515,3587
keywords,salesclerk,helsinki,garbage,independent film,hotel,new year's eve,witch,bet,hotel room,sperm,...,bittersweet,retired army man,affair,closeted homosexual,journalist,newspaper,florida,capitalist,banker,media tycoon
id,3,3,3,3,5,5,5,5,5,5,...,14,14,14,14,15,15,15,15,15,15


## Custom Rating

The value of X percentile for given distribution of vote_count:

In [109]:
percentile = 90

Computing the value of cutoff

In [110]:
pd.read_sql_query("SELECT vote_count AS vote_count_" + str(percentile) + "_percentile FROM (SELECT DISTINCT * FROM MOVIES WHERE imdb_id <> '') ORDER BY vote_count ASC LIMIT 1 OFFSET (SELECT COUNT(*) FROM (SELECT DISTINCT * FROM MOVIES WHERE imdb_id <> '')) * " + str(percentile) + " / 100 - 1;", con)

,vote_count_90_percentile
0,44


Head 10 transposed

In [111]:
pd.read_sql_query("SELECT * FROM (SELECT DISTINCT * FROM MOVIES WHERE imdb_id <> '') WHERE vote_count > (SELECT vote_count AS vote_count_" + str(percentile) + "_percentile FROM (SELECT DISTINCT * FROM MOVIES WHERE imdb_id <> '') ORDER BY vote_count ASC LIMIT 1 OFFSET (SELECT COUNT(*) FROM (SELECT DISTINCT * FROM MOVIES WHERE imdb_id <> '')) * " + str(percentile) + " / 100 - 1) ORDER BY vote_count ASC;" , con).head(10).transpose()

,0,1,2,3,4,5,6,7,8,9
idx,0,0,0,0,0,0,0,0,0,0
id,822,1405,1448,1922,2093,2202,2297,3016,3426,4252
imdb_id,tt0051792,tt0015881,tt0120894,tt0762110,tt0094651,tt0036098,tt0102181,tt0011130,tt0068441,tt0346723
original_language,bn,en,en,en,nl,en,en,en,fr,en
original_title,জলসাঘর,Greed,The Wisdom of Crocodiles,Irina Palm,Amsterdamned,Lassie Come Home,Kafka,Dr. Jekyll and Mr. Hyde,César et Rosalie,Chalte Chalte
popularity,2.148,2.992,3.016,3.243,2.981,3.125,2.836,2.441,5.482,2.915
release_date,1958-10-10,1925-01-26,1998-11-27,2007-02-13,1988-02-11,1943-10-07,1991-11-15,1920-03-18,1972-10-26,2003-06-12
revenue,0,0,0,0,0,0,0,0,0,0
runtime,100,140,98,103,105,88,98,67,110,167
status,Released,Released,Released,Released,Released,Released,Released,Released,Released,Released


Tail 10 transposed

In [112]:
pd.read_sql_query("SELECT * FROM (SELECT DISTINCT * FROM MOVIES WHERE imdb_id <> '') WHERE vote_count > (SELECT vote_count AS vote_count_" + str(percentile) + "_percentile FROM (SELECT DISTINCT * FROM MOVIES WHERE imdb_id <> '') ORDER BY vote_count ASC LIMIT 1 OFFSET (SELECT COUNT(*) FROM (SELECT DISTINCT * FROM MOVIES WHERE imdb_id <> '')) * " + str(percentile) + " / 100 - 1) ORDER BY vote_count ASC;" , con).tail(10).transpose()

,10498,10499,10500,10501,10502,10503,10504,10505,10506,10507
idx,0,0,0,0,0,0,0,0,0,0
id,680,1726,68718,550,118340,157336,19995,155,24428,27205
imdb_id,tt0110912,tt0371746,tt1853728,tt0137523,tt2015381,tt0816692,tt0499549,tt0468569,tt0848228,tt1375666
original_language,en,en,en,en,en,en,en,en,en,en
original_title,Pulp Fiction,Iron Man,Django Unchained,Fight Club,Guardians of the Galaxy,Interstellar,Avatar,The Dark Knight,The Avengers,Inception
popularity,20.07,29.846,20.78,25.452,48.908,34.155,30.333,28.388,46.091,25.264
release_date,1994-09-10,2008-04-30,2012-12-25,1999-10-15,2014-07-30,2014-11-05,2009-12-10,2008-07-16,2012-04-25,2010-07-15
revenue,2.13929e+08,5.85174e+08,4.25368e+08,1.00854e+08,7.73329e+08,6.7512e+08,2.78797e+09,1.00456e+09,1.51956e+09,8.25533e+08
runtime,154,126,165,139,121,169,162,152,143,148
status,Released,Released,Released,Released,Released,Released,Released,Released,Released,Released


Computing Global Average

In [113]:
pd.read_sql_query("SELECT AVG(vote_average) as vote_average_global FROM (SELECT DISTINCT * FROM MOVIES WHERE imdb_id <> '');" , con)

,vote_average_global
0,4.680609


Setting minimum size of vote_count

In [114]:
MinimumVotes = 250.0

Computing Score

Head 10 transposed

In [115]:
pd.read_sql_query("SELECT id, imdb_id, title, vote_count, vote_average, (vote_count / (vote_count + MinimumVotes) * vote_average) + (MinimumVotes / (vote_count + MinimumVotes * VoteAvarageGlobal)) AS Score FROM (SELECT id, imdb_id, title, vote_count, " + str(MinimumVotes) + " as MinimumVotes, vote_average, (SELECT AVG(vote_average) FROM (SELECT DISTINCT * FROM MOVIES WHERE imdb_id <> '')) AS VoteAvarageGlobal FROM (SELECT * FROM (SELECT DISTINCT * FROM MOVIES WHERE imdb_id <> '') WHERE vote_count > (SELECT vote_count AS vote_count_" + str(percentile) + "_percentile FROM (SELECT DISTINCT * FROM MOVIES WHERE imdb_id <> '') ORDER BY vote_count ASC LIMIT 1 OFFSET (SELECT COUNT(*) FROM (SELECT DISTINCT * FROM MOVIES WHERE imdb_id <> '')) * " + str(percentile) + " / 100 - 1) ORDER BY vote_count ASC)) as ParametersForScore ORDER BY Score DESC;" , con).head(10).transpose()

,0,1,2,3,4,5,6,7,8,9
id,278,238,155,550,680,13,122,424,129,240
imdb_id,tt0111161,tt0068646,tt0468569,tt0137523,tt0110912,tt0109830,tt0167260,tt0108052,tt0245429,tt0071562
title,The Shawshank Redemption,The Godfather,The Dark Knight,Fight Club,Pulp Fiction,Forrest Gump,The Lord of the Rings: The Return of the King,Schindler's List,Spirited Away,The Godfather: Part II
vote_count,11681,8943,17110,14598,13443,12939,11829,6958,6453,5205
vote_average,8.6,8.6,8.4,8.4,8.4,8.4,8.4,8.5,8.5,8.5
Score,8.43925,8.39085,8.29271,8.27442,8.26374,8.2585,8.24538,8.23595,8.21577,8.14966


Tail 10 transposed

In [116]:
pd.read_sql_query("SELECT id, imdb_id, title, vote_count, vote_average, (vote_count / (vote_count + MinimumVotes) * vote_average) + (MinimumVotes / (vote_count + MinimumVotes * VoteAvarageGlobal)) AS Score FROM (SELECT id, imdb_id, title, vote_count, " + str(MinimumVotes) + " as MinimumVotes, vote_average, (SELECT AVG(vote_average) FROM (SELECT DISTINCT * FROM MOVIES WHERE imdb_id <> '')) AS VoteAvarageGlobal FROM (SELECT * FROM (SELECT DISTINCT * FROM MOVIES WHERE imdb_id <> '') WHERE vote_count > (SELECT vote_count AS vote_count_" + str(percentile) + "_percentile FROM (SELECT DISTINCT * FROM MOVIES WHERE imdb_id <> '') ORDER BY vote_count ASC LIMIT 1 OFFSET (SELECT COUNT(*) FROM (SELECT DISTINCT * FROM MOVIES WHERE imdb_id <> '')) * " + str(percentile) + " / 100 - 1) ORDER BY vote_count ASC)) as ParametersForScore ORDER BY Score DESC;" , con).tail(10).transpose()

,10498,10499,10500,10501,10502,10503,10504,10505,10506,10507
id,11525,19157,16048,22293,52454,83896,10838,16266,31117,76600
imdb_id,tt0116839,tt0096870,tt0349080,tt0060666,tt1705773,tt1844770,tt0367677,tt1132130,tt0270846,tt1630029
title,Lawnmower Man 2: Beyond Cyberspace,Going Overboard,All About Anna,Manos: The Hands of Fate,Mega Shark vs. Crocosaurus,Sand Sharks,Dracula 3000,2012 Doomsday,Superbabies: Baby Geniuses 2,Avatar: The Way of Water
vote_count,54,57,51,78,55,46,51,47,58,57
vote_average,2.9,2.6,2.8,2,2.6,2.9,2.6,2.5,2.1,0
Score,0.719355,0.68646,0.679143,0.675906,0.672909,0.656242,0.645256,0.60102,0.599012,0.203724


## Text Mining

In [118]:
overview_table = pd.read_sql_query("SELECT DISTINCT O.*, title FROM overview as O LEFT JOIN MoviesDistinct as MD ON O.imdb_id = MD.imdb_id WHERE overview <> ''", con)
overview_table.head

<bound method NDFrame.head of        idx     id    imdb_id  \
0        0      2  tt0094675   
1        0      3  tt0092149   
2        0      5  tt0113101   
3        0      6  tt0107286   
4        0      8  tt0825671   
5        0     11  tt0076759   
6        0     12  tt0266543   
7        0     13  tt0109830   
8        0     14  tt0169547   
9        0     15  tt0033467   
10       0     16  tt0168629   
11       0     17  tt0411267   
12       0     18  tt0119116   
13       0     19  tt0017136   
14       0     20  tt0314412   
15       0     21  tt0060371   
16       0     22  tt0325980   
17       0     24  tt0266697   
18       0     25  tt0418763   
19       0     26  tt0352994   
20       0     27  tt0411705   
21       0     28  tt0078788   
22       0     30  tt1530535   
23       0     31  tt6266826   
24       0     32  tt6264824   
25       0     33  tt0105695   
26       0     35  tt0462538   
27       0     38  tt0338013   
28       0     55  tt0245712   
29       0

In [119]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
overview_table['overview'] = overview_table['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(overview_table['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(14763, 41170)

In [120]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print(cosine_sim)

[[1.         0.01520408 0.         ... 0.         0.         0.        ]
 [0.01520408 1.         0.         ... 0.         0.02298477 0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.00801487 0.        ]
 [0.         0.02298477 0.         ... 0.00801487 1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [122]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(overview_table.index, index=overview_table['title']).drop_duplicates()
indices.head

<bound method NDFrame.head of title
Ariel                                                         0
Shadows in Paradise                                           1
Four Rooms                                                    2
Judgment Night                                                3
Life in Loops (A Megacities RMX)                              4
Star Wars                                                     5
Finding Nemo                                                  6
Forrest Gump                                                  7
American Beauty                                               8
Citizen Kane                                                  9
Dancer in the Dark                                           10
The Dark                                                     11
The Fifth Element                                            12
Metropolis                                                   13
My Life Without Me                                           14
The 

In [123]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[0:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return overview_table['title'].iloc[movie_indices]

In [138]:
get_recommendations('District 9')

10488                                  District 9
6159                                    Meet Dave
12037    Alien Abduction: Incident in Lake County
11686                   Invasion of Astro-Monster
9327                         Godzilla: Final Wars
12433                         Aliens in the Attic
5430                             The High Crusade
14025                  Alien Nation: Dark Horizon
672                             This Island Earth
2029                         Destroy All Monsters
10132                                   Homeworld
Name: title, dtype: object